# Many to One Explainer - Demo of Conjunctive vs Disjunctive Explanations

This demo measures the differences in quality metrics and runtime of explanation generation between conjunctive and disjunctive explanations, on various datasets.\
Each dataset has multiple tests, and at the bottom of each section, the average results are displayed.\
Additionally, at the bottom of the notebook, a collection of all the average results + average across all datasets is displayed.

## Setup

In [1]:
import pandas as pd
import pd_explain
import time
import numpy as np
from sklearn.cluster import KMeans

usetex-False
(CVXPY) Jan 24 07:13:36 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 24 07:13:36 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


This comparison uses a very high, practically unrealistic separation error threshold of 0.8.\
This is set so that less explanations are filtered, and thus we don't get a situation where one form appears better than the other because most of its results were filtered out.\
You can of-course adjust this threshold to a more realistic value, such as 0.2 or 0.3, to get results that are more representative of a real-world scenario.

In [2]:
separation_error = 0.8

By default, we use sampling to speed up the explanation generation process.

In [3]:
pd_explain.get_use_sampling_value()

True

This can be changed either per function call or globally.\
If you would like to run this demo without sampling, uncomment and run the cell below prior to running the rest of the notebook.

In [4]:
# pd_explain.toggle_sampling(False)
# pd_explain.get_use_sampling_value()

In [5]:
def build_comparison_df(comparison_df, test_name, explanation_type, res_df, runtime):
    try:
        # Each group without an explanation has 1 row with "No explanation found" in the 'Explanation' column / index
        num_groups_not_found = res_df.loc[:, 'No explanation found', :].shape[0]
        # Each group that does have explanations can have multiple rows, and we want to count the number of groups, which is the length of the index on the first level
        num_groups_total = len(res_df.index.levels[0])
        percentage_not_found = (num_groups_not_found / num_groups_total) * 100
    # This should only happen in case explanations were found for all instances
    except KeyError:
        percentage_not_found = 0
    try:
        # Get the string representation of the explanations, for each explanation that is not "No explanation found"
        explanations = res_df.index.get_level_values(1)[res_df.index.get_level_values(1) != 'No explanation found'].to_series()
        # Measure the length of each explanation, by counting the number of 'AND' or 'OR' substrings
        explanation_lengths = explanations.apply(lambda x: x.count('AND') + x.count('OR') + 1)
        # Calculate the average explanation length
        avg_explanation_length = explanation_lengths.mean()
        num_explanations_found = len(explanation_lengths)
    # This should only happen in case explanations were not found for any instance
    except (ZeroDivisionError, KeyError):
        avg_explanation_length = pd.NA
        num_explanations_found = 0

    comparison_df.loc[(test_name, explanation_type), :] = [np.mean(res_df['Coverage']), np.mean(res_df['Separation Error']), percentage_not_found, avg_explanation_length, num_explanations_found, runtime]

In [6]:
columns = ['Average Coverage', 'Average Separation Error', '% Explanation Not Found', 'Average Explanation Length', 'Number of Explanations Found', 'Runtime']
index = pd.MultiIndex(levels=[[], []], codes=[[], []], names=['Test', 'Explanation Type'])

## Adults dataset

In [106]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")
adults

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [107]:
adults_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [108]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels='label', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.780376672744751


Coverage  \
Group / Cluster Explanation                                                                      
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-num <= 10                  0.75   
                0 <= capital-gain <= 4243.5                                               0.99   
                1 <= education-num <= 10                                                  0.75   
                0 <= capital-gain <= 5095.5                                                1.0   
>50K            marital-status == Married-civ-spouse AND 8.5 <= education-num <= 16.0     0.81   
                0 <= capital-gain <= 99999                                                 1.0   
                marital-status == Married-civ-spouse                                      0.84   
                8.5 <= education-num <= 16.0                                              0.97   

                                                                                      Separation Error  
Group / Cluster Explanation                                                                             
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-num <= 10                          0.13  
                0 <= capital-gain <= 4243.5                                                        0.2  
                1 <= education-num <= 10                                                          0.15  
                0 <= capital-gain <= 5095.5                                                       0.21  
>50K            marital-status == Married-civ-spouse AND 8.5 <= education-num <= 16.0             0.52  
                0 <= capital-gain <= 99999                                                        0.76  
                marital-status == Married-civ-spouse                                              0.55  
                8.5 <= education-num <= 16.0                                                      0.73

In [109]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels='label', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.0024807453155518


Coverage  \
Group / Cluster Explanation                                                                     
<=50K           1 <= education-num <= 10                                                 0.75   
                0 <= capital-gain <= 5095.5                                               1.0   
                0 <= capital-gain <= 4243.5                                              0.99   
>50K            0 <= capital-gain <= 99999                                                1.0   
                marital-status == Married-civ-spouse                                     0.84   
                8.5 <= education-num <= 16.0                                             0.97   
                8.5 <= education-num <= 16.0 OR marital-status == Married-civ-spouse      1.0   

                                                                                     Separation Error  
Group / Cluster Explanation                                                                            
<=50K           1 <= education-num <= 10                                                         0.15  
                0 <= capital-gain <= 5095.5                                                      0.21  
                0 <= capital-gain <= 4243.5                                                       0.2  
>50K            0 <= capital-gain <= 99999                                                       0.76  
                marital-status == Married-civ-spouse                                             0.55  
                8.5 <= education-num <= 16.0                                                     0.73  
                8.5 <= education-num <= 16.0 OR marital-status == Married-civ-spouse             0.74

### Numeric only and categorical only tests

In [110]:
numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
labels = adults['label']

In [111]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.4226570129394531


Coverage  \
Group / Cluster Explanation                                                         
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-num <= 10     0.75   
                0 <= capital-gain <= 4243.5                                  0.99   
                1 <= education-num <= 10                                     0.75   
                0 <= capital-gain <= 5095.5                                   1.0   
>50K            31.5 <= age <= 90 AND 8.5 <= education-num <= 16.0           0.87   
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90             0.72   
                0 <= capital-gain <= 99999                                    1.0   
                31.5 <= age <= 90                                             0.9   
                37 <= age <= 90                                              0.75   
                8.5 <= education-num <= 16.0                                 0.97   

                                                                         Separation Error  
Group / Cluster Explanation                                                                
<=50K           0 <= capital-gain <= 5095.5 AND 1 <= education-num <= 10             0.13  
                0 <= capital-gain <= 4243.5                                           0.2  
                1 <= education-num <= 10                                             0.15  
                0 <= capital-gain <= 5095.5                                          0.21  
>50K            31.5 <= age <= 90 AND 8.5 <= education-num <= 16.0                   0.64  
                8.5 <= education-num <= 16.0 AND 37 <= age <= 90                     0.62  
                0 <= capital-gain <= 99999                                           0.76  
                31.5 <= age <= 90                                                    0.67  
                37 <= age <= 90                                                      0.65  
                8.5 <= education-num <= 16.0                                         0.73

In [112]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.7917511463165283


Coverage  \
Group / Cluster Explanation                                                
<=50K           1 <= education-num <= 10                            0.75   
                0 <= capital-gain <= 5095.5                          1.0   
                0 <= capital-gain <= 4243.5                         0.99   
>50K            0 <= capital-gain <= 99999                           1.0   
                31.5 <= age <= 90                                    0.9   
                8.5 <= education-num <= 16.0                        0.97   
                8.5 <= education-num <= 16.0 OR 37 <= age <= 90      1.0   
                37 <= age <= 90                                     0.75   

                                                                Separation Error  
Group / Cluster Explanation                                                       
<=50K           1 <= education-num <= 10                                    0.15  
                0 <= capital-gain <= 5095.5                                 0.21  
                0 <= capital-gain <= 4243.5                                  0.2  
>50K            0 <= capital-gain <= 99999                                  0.76  
                31.5 <= age <= 90                                           0.67  
                8.5 <= education-num <= 16.0                                0.73  
                8.5 <= education-num <= 16.0 OR 37 <= age <= 90             0.74  
                37 <= age <= 90                                             0.65

In [113]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.2548182010650635


Coverage  \
Group / Cluster Explanation                                                               
<=50K           occupation != Prof-specialty AND occupation != Exec-managerial     0.82   
                occupation != Prof-specialty                                       0.91   
                occupation != Exec-managerial                                      0.91   
>50K            marital-status == Married-civ-spouse                               0.84   

                                                                               Separation Error  
Group / Cluster Explanation                                                                      
<=50K           occupation != Prof-specialty AND occupation != Exec-managerial             0.17  
                occupation != Prof-specialty                                               0.21  
                occupation != Exec-managerial                                              0.21  
>50K            marital-status == Married-civ-spouse                                       0.55

In [114]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.3261420726776123


Coverage  \
Group / Cluster Explanation                                                              
<=50K           occupation != Exec-managerial                                     0.91   
                occupation != Exec-managerial OR occupation != Prof-specialty      1.0   
                occupation != Prof-specialty                                      0.91   
>50K            marital-status == Married-civ-spouse                              0.84   

                                                                              Separation Error  
Group / Cluster Explanation                                                                     
<=50K           occupation != Exec-managerial                                             0.21  
                occupation != Exec-managerial OR occupation != Prof-specialty             0.24  
                occupation != Prof-specialty                                              0.21  
>50K            marital-status == Married-civ-spouse                                      0.55

### Groupby tests

In [115]:
single_attribute_groupby = adults.groupby('relationship').mean()
multi_attribute_groupby = adults.groupby(['sex', 'relationship']).mean()

In [116]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 3.3704495429992676


Coverage  \
Group / Cluster Explanation                                                                          
Not-in-family   marital-status != Married-civ-spouse AND 20.5 <= age <= 90                    0.96   
                marital-status != Married-civ-spouse AND 23.5 <= age <= 90                    0.88   
                marital-status != Married-civ-spouse                                           1.0   
Husband         marital-status == Married-civ-spouse                                           1.0   
                marital-status == Married-civ-spouse AND sex != Female                         1.0   
                marital-status == Married-civ-spouse AND sex == Male                           1.0   
Wife            marital-status == Married-civ-spouse AND sex == Female                        0.99   
Own-child       marital-status != Married-civ-spouse AND 17.0 <= age <= 31.6                  0.82   
                17 <= age <= 31 AND marital-status == Never-married                           0.78   
                marital-status != Married-civ-spouse                                          0.98   
                17.0 <= age <= 31.6 AND marital-status == Never-married                       0.78   
                17.0 <= age <= 35.25 AND marital-status == Never-married                      0.82   
                marital-status != Married-civ-spouse AND 17 <= age <= 28                      0.76   
                17.0 <= age <= 53.5 AND marital-status == Never-married                       0.88   
                marital-status != Married-civ-spouse AND 17.0 <= age <= 53.5                  0.98   
                17 <= age <= 37 AND marital-status == Never-married                           0.84   
                marital-status == Never-married                                               0.89   
                17.0 <= age <= 41.33333333333333 AND marital-status == Never-married          0.86   
                marital-status != Married-civ-spouse AND 17.0 <= age <= 35.25                 0.88   
                17 <= age <= 28 AND marital-status == Never-married                           0.73   
                marital-status != Married-civ-spouse AND 17.0 <= age <= 41.33333333333333     0.94   
                marital-status != Married-civ-spouse AND 17 <= age <= 37                       0.9   
                17 <= age <= 28                                                               0.76   
                marital-status != Married-civ-spouse AND 17 <= age <= 31                      0.82   
Unmarried       marital-status != Married-civ-spouse AND 24.5 <= age <= 90                    0.94   
                marital-status != Married-civ-spouse AND 23.5 <= age <= 90                    0.95   
                marital-status != Married-civ-spouse AND marital-status != Never-married      0.76   
                marital-status != Married-civ-spouse AND 27.5 <= age <= 90                    0.88   
Other-relative  No explanation found                                                           NaN   

                                                                                          Separation Error  
Group / Cluster Explanation                                                                                 
Not-in-family   marital-status != Married-civ-spouse AND 20.5 <= age <= 90                            0.47  
                marital-status != Married-civ-spouse AND 23.5 <= age <= 90                            0.44  
                marital-status != Married-civ-spouse                                                  0.52  
Husband         marital-status == Married-civ-spouse                                                  0.11  
                marital-status == Married-civ-spouse AND sex != Female                                0.01  
                marital-status == Married-civ-spouse AND sex == Male                                  0.01  
Wife            marital-status == Married-civ-spouse AN

In [117]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.6502394676208496


Coverage  \
Group / Cluster Explanation                                                                         
Not-in-family   marital-status != Married-civ-spouse                                          1.0   
Husband         marital-status == Married-civ-spouse                                          1.0   
Own-child       marital-status != Married-civ-spouse                                         0.98   
                17 <= age <= 28 OR marital-status == Never-married                           0.92   
                marital-status != Married-civ-spouse OR 17.0 <= age <= 41.33333333333333      1.0   
                17 <= age <= 31 OR marital-status == Never-married                           0.93   
                17 <= age <= 28                                                              0.76   
                marital-status != Married-civ-spouse OR 17 <= age <= 28                      0.99   
                marital-status == Never-married                                              0.89   
                marital-status == Never-married OR 17.0 <= age <= 31.6                       0.93   
Other-relative  No explanation found                                                          NaN   
Unmarried       No explanation found                                                          NaN   
Wife            No explanation found                                                          NaN   

                                                                                         Separation Error  
Group / Cluster Explanation                                                                                
Not-in-family   marital-status != Married-civ-spouse                                                 0.52  
Husband         marital-status == Married-civ-spouse                                                 0.11  
Own-child       marital-status != Married-civ-spouse                                                 0.72  
                17 <= age <= 28 OR marital-status == Never-married                                   0.64  
                marital-status != Married-civ-spouse OR 17.0 <= age <= 41.33333333333333              0.8  
                17 <= age <= 31 OR marital-status == Never-married                                   0.68  
                17 <= age <= 28                                                                      0.56  
                marital-status != Married-civ-spouse OR 17 <= age <= 28                              0.74  
                marital-status == Never-married                                                      0.59  
                marital-status == Never-married OR 17.0 <= age <= 31.6                               0.68  
Other-relative  No explanation found                                                                  NaN  
Unmarried       No explanation found                                                                  NaN  
Wife            No explanation found                                                                  NaN

In [118]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 7.376740217208862


Coverage  \
Group / Cluster              Explanation                                                                           
('Male', 'Not-in-family')    marital-status != Married-civ-spouse AND 26.5 <= age <= 90                     0.78   
                             marital-status != Married-civ-spouse AND 20.5 <= age <= 90                     0.97   
                             marital-status != Married-civ-spouse                                            1.0   
                             marital-status != Married-civ-spouse AND 23.5 <= age <= 90                     0.89   
('Male', 'Husband')          marital-status == Married-civ-spouse AND 39.5 <= hours-per-week <= 99          0.88   
                             marital-status == Married-civ-spouse AND 40 <= hours-per-week <= 99            0.88   
                             marital-status == Married-civ-spouse AND occupation != Adm-clerical            0.95   
                             marital-status == Married-civ-spouse                                            1.0   
                             marital-status == Married-civ-spouse AND 38.5 <= hours-per-week <= 99          0.88   
('Female', 'Not-in-family')  marital-status != Married-civ-spouse AND 26.5 <= age <= 90                     0.79   
                             marital-status != Married-civ-spouse                                            1.0   
                             marital-status != Married-civ-spouse AND 23.5 <= age <= 90                     0.87   
                             marital-status != Married-civ-spouse AND occupation != Craft-repair            0.98   
('Female', 'Own-child')      17.0 <= age <= 29.166666666666664 AND marital-status != Married-civ-spouse     0.79   
                             17.0 <= age <= 29.166666666666664                                              0.79   
                             17.0 <= age <= 31.6 AND marital-status != Married-civ-spouse                   0.82   
                             17.0 <= age <= 41.33333333333333 AND marital-status == Never-married           0.85   
                             17 <= age <= 31 AND marital-status != Married-civ-spouse                       0.82   
                             17 <= age <= 28 AND marital-status == Never-married                            0.73   
                             17.0 <= age <= 35.25 AND marital-status == Never-married                       0.82   
                             17.0 <= age <= 29.166666666666664 AND marital-status == Never-married          0.76   
                             17 <= age <= 26                                                                 0.7   
                             17.0 <= age <= 53.5 AND marital-status == Never-married                        0.87   
('Male', 'Own-child')        17 <= age <= 28                                                                0.75   
                             17 <= age <= 37 AND marital-status == Never-married                            0.83   
                             17 <= age <= 31 AND marital-status == Never-married                            0.77   
                             17.0 <= age <= 41.33333333333333 AND marital-status == Never-married           0.85   
                             17 <= age <= 28 AND marital-status == Never-married                            0.73   
                             17.0 <= age <= 31.6 AND marital-status == Never-married                        0.77   
                             marital-status == Never-married                                                0.88   
                             17.0 <= age <= 29.166666666666664 AND marital-status == Never-married          0.74   
                             17.0 <= age <= 41.33333333333333 AND marital-status != Married-civ-spouse      0.92   
                             17.0 <= age <= 35.25 AND marital-status == Never-married                       0.82   
('Female', 'Unmarried')      marital-status 

In [119]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 7.442141771316528


Coverage  \
Group / Cluster              Explanation                                                   
('Male', 'Not-in-family')    marital-status != Married-civ-spouse                    1.0   
('Male', 'Husband')          marital-status == Married-civ-spouse                    1.0   
('Female', 'Not-in-family')  marital-status != Married-civ-spouse                    1.0   
('Female', 'Own-child')      17.0 <= age <= 29.166666666666664                      0.79   
                             17 <= age <= 26                                         0.7   
('Male', 'Own-child')        marital-status == Never-married                        0.88   
                             17 <= age <= 28 OR marital-status == Never-married     0.91   
                             17 <= age <= 28                                        0.75   
('Female', 'Other-relative') No explanation found                                    NaN   
('Female', 'Unmarried')      No explanation found                                    NaN   
('Female', 'Wife')           No explanation found                                    NaN   
('Male', 'Other-relative')   No explanation found                                    NaN   
('Male', 'Unmarried')        No explanation found                                    NaN   

                                                                                Separation Error  
Group / Cluster              Explanation                                                          
('Male', 'Not-in-family')    marital-status != Married-civ-spouse                           0.74  
('Male', 'Husband')          marital-status == Married-civ-spouse                           0.11  
('Female', 'Not-in-family')  marital-status != Married-civ-spouse                           0.78  
('Female', 'Own-child')      17.0 <= age <= 29.166666666666664                               0.8  
                             17 <= age <= 26                                                0.76  
('Male', 'Own-child')        marital-status == Never-married                                0.77  
                             17 <= age <= 28 OR marital-status == Never-married              0.8  
                             17 <= age <= 28                                                0.76  
('Female', 'Other-relative') No explanation found                                            NaN  
('Female', 'Unmarried')      No explanation found                                            NaN  
('Female', 'Wife')           No explanation found                                            NaN  
('Male', 'Other-relative')   No explanation found                                            NaN  
('Male', 'Unmarried')        No explanation found                                            NaN

### Clustering test

#### 3 clusters

In [120]:
cluster_labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(adults))

In [121]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.1290814876556396


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,291277.5 <= fnlwgt <= 1490400,1.0,0.0
1,13769 <= fnlwgt <= 149278.5,1.0,0.0
2,149278.5 <= fnlwgt <= 291277.5,1.0,0.0


In [122]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 2.4612245559692383


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,291277.5 <= fnlwgt <= 1490400,1.0,0.0
1,13769 <= fnlwgt <= 149278.5,1.0,0.0
2,149278.5 <= fnlwgt <= 291277.5,1.0,0.0


#### 15 clusters

In [123]:
cluster_labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(adults))

In [124]:
start_time = time.time()
conj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj', separation_threshold=separation_error)
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 17.394453048706055


Coverage  \
Group / Cluster Explanation                                                                          
1               175034 <= fnlwgt <= 223242                                                     1.0   
                175034 <= fnlwgt <= 223242 AND 8.5 <= education-num <= 16.0                   0.87   
                175034 <= fnlwgt <= 223242 AND 8.5 <= education-num <= 13.5                   0.79   
                175034 <= fnlwgt <= 202206                                                    0.72   
2               126259.0 <= fnlwgt <= 156449.5                                                 1.0   
4               281883.0 <= fnlwgt <= 371408                                                   1.0   
                281883.0 <= fnlwgt <= 371408 AND 8.5 <= education-num <= 13.5                  0.8   
5               19302 <= fnlwgt <= 58238.0                                                     1.0   
6               202206 <= fnlwgt <= 253370                                                     1.0   
                202206 <= fnlwgt <= 253370 AND 1 <= education-num <= 10                       0.71   
                202206 <= fnlwgt <= 253370 AND 1.5 <= education-num <= 10.5                   0.71   
10              230565.59999999998 <= fnlwgt <= 300986.8 AND 5.5 <= education-num <= 13.5     0.89   
                230565.59999999998 <= fnlwgt <= 300986.8                                       1.0   
11              323591.5 <= fnlwgt <= 371408                                                   1.0   
12              156449.5 <= fnlwgt <= 183996.5                                                 1.0   
13              93957.5 <= fnlwgt <= 126259.0                                                  1.0   
14              58238.0 <= fnlwgt <= 93957.5                                                   1.0   

                                                                                          Separation Error  
Group / Cluster Explanation                                                                                 
1               175034 <= fnlwgt <= 223242                                                            0.38  
                175034 <= fnlwgt <= 223242 AND 8.5 <= education-num <= 16.0                           0.37  
                175034 <= fnlwgt <= 223242 AND 8.5 <= education-num <= 13.5                           0.36  
                175034 <= fnlwgt <= 202206                                                            0.33  
2               126259.0 <= fnlwgt <= 156449.5                                                         0.0  
4               281883.0 <= fnlwgt <= 371408                                                          0.43  
                281883.0 <= fnlwgt <= 371408 AND 8.5 <= education-num <= 13.5                         0.42  
5               19302 <= fnlwgt <= 58238.0                                                             0.0  
6               202206 <= fnlwgt <= 253370                                                            0.39  
                202206 <= fnlwgt <= 253370 AND 1 <= education-num <= 10                               0.38  
                202206 <= fnlwgt <= 253370 AND 1.5 <= education-num <= 10.5                           0.38  
10              230565.59999999998 <= fnlwgt <= 300986.8 AND 5.5 <= education-num <= 13.5             0.49  
                230565.59999999998 <= fnlwgt <= 300986.8                                               0.5  
11              323591.5 <= fnlwgt <= 371408                                                           0.0  
12              156449.5 <= fnlwgt <= 183996.5                                                         0.0  
13              93957.5 <= fnlwgt <= 126259.0                                                          0.0  
14              58238.0 <= fnlwgt <= 93957.5                                                           0.0

In [125]:
start_time = time.time()
disj_res = adults.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj', separation_threshold=separation_error)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(adults_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 19.559987545013428


Coverage  \
Group / Cluster Explanation                                                  
1               175034 <= fnlwgt <= 223242                             1.0   
                175034 <= fnlwgt <= 202206                            0.72   
2               126259.0 <= fnlwgt <= 156449.5                         1.0   
4               281883.0 <= fnlwgt <= 371408                           1.0   
5               19302 <= fnlwgt <= 58238.0                             1.0   
6               202206 <= fnlwgt <= 253370                             1.0   
10              254039.33333333334 <= fnlwgt <= 312723.6666666667      0.7   
                230565.59999999998 <= fnlwgt <= 300986.8               1.0   
11              323591.5 <= fnlwgt <= 371408                           1.0   
12              156449.5 <= fnlwgt <= 183996.5                         1.0   
13              93957.5 <= fnlwgt <= 126259.0                          1.0   
14              58238.0 <= fnlwgt <= 93957.5                           1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
1               175034 <= fnlwgt <= 223242                                    0.38  
                175034 <= fnlwgt <= 202206                                    0.33  
2               126259.0 <= fnlwgt <= 156449.5                                 0.0  
4               281883.0 <= fnlwgt <= 371408                                  0.43  
5               19302 <= fnlwgt <= 58238.0                                     0.0  
6               202206 <= fnlwgt <= 253370                                    0.39  
10              254039.33333333334 <= fnlwgt <= 312723.6666666667             0.49  
                230565.59999999998 <= fnlwgt <= 300986.8                       0.5  
11              323591.5 <= fnlwgt <= 371408                                   0.0  
12              156449.5 <= fnlwgt <= 183996.5                                 0.0  
13              93957.5 <= fnlwgt <= 126259.0                                  0.0  
14              58238.0 <= fnlwgt <= 93957.5                                   0.0

### Summary of results

In [126]:
conj_averages = adults_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = adults_comparison_df.loc[:, 'disjunctive', :].mean()
adults_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
adults_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [127]:
adults_comparison_df

Average Coverage  \
Test                     Explanation Type                      
Full dataset             conjunctive                 0.88875   
                         disjunctive      0.9357142857142857   
Numeric only             conjunctive      0.8700000000000001   
                         disjunctive                    0.92   
Categorical only         conjunctive                    0.87   
                         disjunctive                   0.915   
Single attribute groupby conjunctive      0.8850000000000001   
                         disjunctive      0.9400000000000001   
Multi attribute groupby  conjunctive      0.8454285714285714   
                         disjunctive                 0.87875   
3 clusters               conjunctive                     1.0   
                         disjunctive                     1.0   
15 clusters              conjunctive      0.9111764705882354   
                         disjunctive      0.9516666666666667   
Average                  conjunctive       0.895765006002401   
                         disjunctive      0.9344472789115645   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                       0.40625   
                         disjunctive           0.47714285714285715   
Numeric only             conjunctive                         0.476   
                         disjunctive                       0.51375   
Categorical only         conjunctive           0.28500000000000003   
                         disjunctive                        0.3025   
Single attribute groupby conjunctive            0.4996428571428571   
                         disjunctive                         0.604   
Multi attribute groupby  conjunctive            0.6571428571428571   
                         disjunctive                          0.69   
3 clusters               conjunctive                           0.0   
                         disjunctive                           0.0   
15 clusters              conjunctive            0.2605882352941176   
                         disjunctive           0.20999999999999996   
Average                  conjunctive            0.3692319927971189   
                         disjunctive           0.39962755102040814   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                            0   
                         disjunctive                            0   
Numeric only             conjunctive                            0   
                         disjunctive                            0   
Categorical only         conjunctive                            0   
                         disjunctive                            0   
Single attribute groupby conjunctive           16.666666666666664   
                         disjunctive                         50.0   
Multi attribute groupby  conjunctive                         40.0   
                         disjunctive                         50.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                            0   
                         disjunctive                            0   
Average                  conjunctive            8.095238095238095   
                         disjunctive           14.285714285714286   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive                            1.25   
                         disjunctive              1.1428571428571428   
Numeric only             conjunctive                             1.3   
                         di

## Spotify dataset

In [128]:
spotify = pd.read_csv(r"..\Examples\Datasets\spotify_all.csv")
spotify

,acousticness,artists,danceability,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,speechiness,tempo,valence,year,decade,popularity_score,main_artist,duration_minutes
0,0.991,['Mamie Smith'],0.598,0.2239999999999999,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.379,-12.628,...,Keep A Song In Your Soul,12,0.0936,149.976,0.634,1920,1920,10,Mamie Smith,2.80555
1,0.643,"[""Screamin' Jay Hawkins""]",0.852,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.0264,5,0.0809,-7.261,...,I Put A Spell On You,7,0.0534,86.889,0.95,1920,1920,0,Screamin' Jay Hawkins,2.5033333333333334
2,0.993,['Mamie Smith'],0.647,0.186,0,11m7laMUgmOKqI3oYzuhne,1.76e-05,0,0.519,-12.098,...,Golfing Papa,4,0.174,97.6,0.6890000000000001,1920,1920,0,Mamie Smith,2.73045
3,0.000173,['Oscar Velazquez'],0.73,0.7979999999999999,0,19Lc5SfJJ5O1oaxY0fpwfh,0.8009999999999999,2,0.128,-7.311,...,True House Music - Xavier Santos & Carlos Gomix Remix,17,0.0425,127.997,0.0422,1920,1920,10,Oscar Velazquez,7.034783333333333
4,0.295,['Mixe'],0.7040000000000001,0.7070000000000001,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.402,-6.0360000000000005,...,Xuniverxe,2,0.0768,122.076,0.299,1920,1920,0,Mixe,2.7537333333333334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.00917,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,0.866,0,46LhBf6TvYjZU2SMvGZAbn,5.99e-05,6,0.178,-5.089,...,The One,0,0.0356,125.972,0.186,2020,2020,0,DJ Combo,2.46025
174385,0.795,['Alessia Cara'],0.429,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.0,4,0.196,-11.665,...,A Little More,0,0.036,94.71,0.228,2021,2020,0,Alessia Cara,2.412
174386,0.8059999999999999,['Roger Fly'],0.6709999999999999,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.92,4,0.113,-12.393,...,Together,0,0.0282,108.058,0.7140000000000001,2020,2020,0,Roger Fly,3.635783333333333
174387,0.92,['Taylor Swift'],0.462,0.24,1,1gcyHQpBQ1lfXGdhZmWrHP,0.0,0,0.113,-12.077,...,champagne problems,69,0.0377,171.31900000000005,0.32,2021,2020,60,Taylor Swift,4.066666666666666


In [129]:
spotify_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [130]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.2431085109710693


Coverage  \
Group / Cluster Explanation                                                                     
0               0.0 <= speechiness <= 0.2425                                             0.94   
                0.0 <= speechiness <= 0.194 AND 0 <= popularity <= 46.5                  0.75   
                0.0 <= speechiness <= 0.3233333333333333 AND 0 <= popularity <= 46.5     0.78   
                0.0 <= speechiness <= 0.3233333333333333                                 0.95   
                0.0 <= speechiness <= 0.16166666666666665                                0.91   
                0.0 <= speechiness <= 0.485                                              0.97   
1               No explanation found                                                      NaN   

                                                                                     Separation Error  
Group / Cluster Explanation                                                                            
0               0.0 <= speechiness <= 0.2425                                                     0.04  
                0.0 <= speechiness <= 0.194 AND 0 <= popularity <= 46.5                          0.01  
                0.0 <= speechiness <= 0.3233333333333333 AND 0 <= popularity <= 46.5             0.02  
                0.0 <= speechiness <= 0.3233333333333333                                         0.05  
                0.0 <= speechiness <= 0.16166666666666665                                        0.03  
                0.0 <= speechiness <= 0.485                                                      0.06  
1               No explanation found                                                              NaN

In [131]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels='explicit', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 2.6184418201446533


Coverage  \
Group / Cluster Explanation                                                                       
0               0.0 <= speechiness <= 0.3233333333333333                                   0.95   
                0 <= popularity <= 46.5 OR 0.0 <= speechiness <= 0.16166666666666665       0.99   
                0.0 <= speechiness <= 0.2425 OR 0.0 <= popularity <= 44.0                   1.0   
                0.0 <= speechiness <= 0.485                                                0.97   
                0.0 <= speechiness <= 0.16166666666666665                                  0.91   
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness <= 0.16166666666666665     0.99   
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness <= 0.194                   0.99   
                0.0 <= speechiness <= 0.2425                                               0.94   
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity <= 46.5                     1.0   
1               No explanation found                                                        NaN   

                                                                                       Separation Error  
Group / Cluster Explanation                                                                              
0               0.0 <= speechiness <= 0.3233333333333333                                           0.05  
                0 <= popularity <= 46.5 OR 0.0 <= speechiness <= 0.16166666666666665               0.05  
                0.0 <= speechiness <= 0.2425 OR 0.0 <= popularity <= 44.0                          0.06  
                0.0 <= speechiness <= 0.485                                                        0.06  
                0.0 <= speechiness <= 0.16166666666666665                                          0.03  
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness <= 0.16166666666666665             0.05  
                0.0 <= popularity <= 44.0 OR 0.0 <= speechiness <= 0.194                           0.05  
                0.0 <= speechiness <= 0.2425                                                       0.04  
                0.0 <= speechiness <= 0.2425 OR 0 <= popularity <= 46.5                            0.06  
1               No explanation found                                                                NaN

### Numeric only and categorical only tests

In [132]:
numeric_only = spotify[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']]
categorical_only = spotify[['name', 'main_artist', 'year']]
labels = spotify['explicit']

In [133]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.027392625808716


Coverage  \
Group / Cluster Explanation                                          
0               0.0 <= speechiness <= 0.2425                  0.94   
                0.0 <= speechiness <= 0.16166666666666665     0.91   
                0.0 <= speechiness <= 0.3233333333333333      0.95   
                0.0 <= speechiness <= 0.485                   0.97   
1               No explanation found                           NaN   

                                                          Separation Error  
Group / Cluster Explanation                                                 
0               0.0 <= speechiness <= 0.2425                          0.04  
                0.0 <= speechiness <= 0.16166666666666665             0.03  
                0.0 <= speechiness <= 0.3233333333333333              0.05  
                0.0 <= speechiness <= 0.485                           0.06  
1               No explanation found                                   NaN

In [134]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.8733229637145996


Coverage  \
Group / Cluster Explanation                                                                              
0               0.0 <= speechiness <= 0.3233333333333333                                          0.95   
                0.0 <= speechiness <= 0.485                                                       0.97   
                0.0 <= speechiness <= 0.485 OR 0.0 <= instrumentalness <= 0.4995                   1.0   
                0.0 <= speechiness <= 0.16166666666666665                                         0.91   
                0.0 <= speechiness <= 0.2425                                                      0.94   
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= speechiness <= 0.3233333333333333       1.0   
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= speechiness <= 0.485                  1.0   
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= speechiness <= 0.485                    1.0   
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0 <= instrumentalness <= 0.4995      1.0   
1               No explanation found                                                               NaN   

                                                                                              Separation Error  
Group / Cluster Explanation                                                                                     
0               0.0 <= speechiness <= 0.3233333333333333                                                  0.05  
                0.0 <= speechiness <= 0.485                                                               0.06  
                0.0 <= speechiness <= 0.485 OR 0.0 <= instrumentalness <= 0.4995                          0.07  
                0.0 <= speechiness <= 0.16166666666666665                                                 0.03  
                0.0 <= speechiness <= 0.2425                                                              0.04  
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= speechiness <= 0.3233333333333333              0.07  
                0.0 <= instrumentalness <= 0.24975 OR 0.0 <= speechiness <= 0.485                         0.07  
                0.0 <= instrumentalness <= 0.333 OR 0.0 <= speechiness <= 0.485                           0.07  
                0.0 <= speechiness <= 0.3233333333333333 OR 0.0 <= instrumentalness <= 0.4995             0.07  
1               No explanation found                                                                       NaN

In [135]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 0.7465677261352539


Coverage  \
Group / Cluster Explanation                                           
0               name == infrequent_sklearn                      1.0   
                main_artist == artist_infrequent_sklearn        1.0   
1               No explanation found                            NaN   

                                                           Separation Error  
Group / Cluster Explanation                                                  
0               name == infrequent_sklearn                             0.07  
                main_artist == artist_infrequent_sklearn               0.07  
1               No explanation found                                    NaN

In [136]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.8602464199066162


Coverage  \
Group / Cluster Explanation                                         
0               name == infrequent_sklearn                    1.0   
                main_artist == artist_infrequent_sklearn      1.0   
1               No explanation found                          NaN   

                                                         Separation Error  
Group / Cluster Explanation                                                
0               name == infrequent_sklearn                           0.07  
                main_artist == artist_infrequent_sklearn             0.07  
1               No explanation found                                  NaN

### Groupby tests

In [137]:
single_attribute_groupby = spotify.groupby('mode').mean()

In [138]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 2.795407295227051


Coverage  \
Group / Cluster Explanation                                                                                          
1               0.07324999943375587 <= acousticness <= 0.996                                                  0.78   
                0.0 <= speechiness <= 0.3233333333333333 AND 0.07324999943375587 <= acousticness <= 0.996     0.73   
                0.0 <= speechiness <= 0.485                                                                   0.95   
0               No explanation found                                                                           NaN   

                                                                                                          Separation Error  
Group / Cluster Explanation                                                                                                 
1               0.07324999943375587 <= acousticness <= 0.996                                                          0.28  
                0.0 <= speechiness <= 0.3233333333333333 AND 0.07324999943375587 <= acousticness <= 0.996             0.27  
                0.0 <= speechiness <= 0.485                                                                            0.3  
0               No explanation found                                                                                   NaN

In [139]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.223463296890259


Coverage  \
Group / Cluster Explanation                                                                            
1               0.0 <= speechiness <= 0.485 OR 0.07324999943375587 <= acousticness <= 0.996      1.0   
                0.07324999943375587 <= acousticness <= 0.996                                    0.78   
                0.0 <= speechiness <= 0.485                                                     0.95   
0               No explanation found                                                             NaN   

                                                                                            Separation Error  
Group / Cluster Explanation                                                                                   
1               0.0 <= speechiness <= 0.485 OR 0.07324999943375587 <= acousticness <= 0.996              0.3  
                0.07324999943375587 <= acousticness <= 0.996                                            0.28  
                0.0 <= speechiness <= 0.485                                                              0.3  
0               No explanation found                                                                     NaN

### Clustering test

#### 3 clusters

In [140]:
# The spotify dataset is too big to cluster all columns with get_dummies (I tried, got a MemoryError, saying it is unable to allocate 22.3 GiB)
cols = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

In [141]:
cluster_labels = KMeans(n_clusters=3).fit_predict(spotify[cols])

In [142]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 3.391819953918457


Coverage  \
Group / Cluster Explanation                                                  
0               147.00550079345703 <= tempo <= 219.446                 1.0   
1               0.0 <= tempo <= 106.19950103759766                    0.99   
                0.0 <= tempo <= 109.723                                1.0   
                0.0 <= tempo <= 106.32799911499023                    0.99   
2               106.73249816894531 <= tempo <= 147.00550079345703      1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               147.00550079345703 <= tempo <= 219.446                         0.0  
1               0.0 <= tempo <= 106.19950103759766                             0.0  
                0.0 <= tempo <= 109.723                                       0.08  
                0.0 <= tempo <= 106.32799911499023                             0.0  
2               106.73249816894531 <= tempo <= 147.00550079345703              0.0

In [143]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.8467187881469727


Coverage  \
Group / Cluster Explanation                                                  
0               147.00550079345703 <= tempo <= 219.446                 1.0   
1               0.0 <= tempo <= 109.723                                1.0   
                0.0 <= tempo <= 106.32799911499023                    0.99   
                0.0 <= tempo <= 106.19950103759766                    0.99   
2               106.73249816894531 <= tempo <= 147.00550079345703      1.0   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
0               147.00550079345703 <= tempo <= 219.446                         0.0  
1               0.0 <= tempo <= 109.723                                       0.08  
                0.0 <= tempo <= 106.32799911499023                             0.0  
                0.0 <= tempo <= 106.19950103759766                             0.0  
2               106.73249816894531 <= tempo <= 147.00550079345703              0.0

#### 15 clusters

In [144]:
cluster_labels = KMeans(n_clusters=15).fit_predict(spotify[cols])

In [145]:
start_time = time.time()
conj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 12.207214593887329


Coverage  \
Group / Cluster Explanation                                                                                                               
2               70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -13.771 AND 0.05980000086128712 <= acousticness <= 0.996      0.99   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -15.783                                                       0.91   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -15.783 AND 0.05980000086128712 <= acousticness <= 0.996       0.9   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -14.898 AND 0.058150000870227814 <= acousticness <= 0.996     0.96   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -15.783 AND 0.058150000870227814 <= acousticness <= 0.996      0.9   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -14.898 AND 0.05980000086128712 <= acousticness <= 0.996      0.96   
                70.283 <= tempo <= 90.657 AND -43.738 <= loudness <= -15.783                                                       0.82   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -13.771                                                        1.0   
                70.283 <= tempo <= 91.1602 AND -43.738 <= loudness <= -15.783                                                      0.84   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -14.898                                                       0.97   
                70.283 <= tempo <= 94.278 AND -43.738 <= loudness <= -13.771 AND 0.058150000870227814 <= acousticness <= 0.996     0.99   
3               113.077 <= tempo <= 125.01                                                                                         0.92   
                113.077 <= tempo <= 125.01 AND -21.7975 <= loudness <= 0.14300000000000068                                         0.92   
                113.077 <= tempo <= 125.01 AND -21.5310001373291 <= loudness <= 0.143                                              0.92   
4               120.076 <= tempo <= 136.47 AND -14.483999999999998 <= loudness <= 0.14300000000000068                               1.0   
                120.076 <= tempo <= 136.47 AND -10.82725 <= loudness <= 0.14300000000000068                                        0.77   
                120.076 <= tempo <= 136.47                                                                                          1.0   
                120.076 <= tempo <= 136.47 AND -13.44249963760376 <= loudness <= 0.143                                             0.98   
5               86.92000198364258 <= tempo <= 100.5 AND -21.7975 <= loudness <= 0.14300000000000068                                 1.0   
                86.92000198364258 <= tempo <= 100.5 AND -14.483999999999998 <= loudness <= 0.14300000000000068                     0.89   
                86.92000198364258 <= tempo <= 100.5620002746582 AND -15.234000205993652 <= loudness <= 0.143                       0.93   
                86.92000198364258 <= tempo <= 100.5620002746582 AND -21.7975 <= loudness <= 0.14300000000000068                     1.0   
                87.855 <= tempo <= 100.062                                                                                         0.93   
                86.92000198364258 <= tempo <= 100.5620002746582                                                                     1.0   
                86.92000198364258 <= tempo <= 100.5 AND -18.134499549865723 <= loudness <= 0.143                                   0.99   
                87.855 <= tempo <= 100.062 AND -13.613000392913818 <= loudness <= -4.851999998092651                               0.71   
                87.855 <= tempo <= 100.062 AND -15.234000205993652 <= loudness <= 0.143                                            0.86   
                86.92000198364258 <= tempo <= 100.5620002746582 AND -18.134499549865723 <= loudn

In [146]:
start_time = time.time()
disj_res = spotify.explain(explainer='many to one', labels=cluster_labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(spotify_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 10.725688457489014


Coverage  \
Group / Cluster Explanation                                                  
3               113.077 <= tempo <= 125.01                            0.92   
4               120.076 <= tempo <= 136.47                             1.0   
5               86.92000198364258 <= tempo <= 100.5                    1.0   
                86.92000198364258 <= tempo <= 100.5620002746582        1.0   
                87.855 <= tempo <= 100.062                            0.93   
6               160.1125030517578 <= tempo <= 174.669                 0.98   
                157.27133333333336 <= tempo <= 174.66900000000004      1.0   
9               100.5 <= tempo <= 112.34099960327148                  0.98   
                100.5359992980957 <= tempo <= 112.48799896240234      0.99   
10              132.9146 <= tempo <= 153.79180000000002                1.0   
1               No explanation found                                   NaN   
2               No explanation found                                   NaN   
7               No explanation found                                   NaN   
12              No explanation found                                   NaN   

                                                                  Separation Error  
Group / Cluster Explanation                                                         
3               113.077 <= tempo <= 125.01                                    0.21  
4               120.076 <= tempo <= 136.47                                     0.3  
5               86.92000198364258 <= tempo <= 100.5                           0.22  
                86.92000198364258 <= tempo <= 100.5620002746582               0.22  
                87.855 <= tempo <= 100.062                                    0.19  
6               160.1125030517578 <= tempo <= 174.669                         0.01  
                157.27133333333336 <= tempo <= 174.66900000000004             0.18  
9               100.5 <= tempo <= 112.34099960327148                          0.26  
                100.5359992980957 <= tempo <= 112.48799896240234              0.27  
10              132.9146 <= tempo <= 153.79180000000002                        0.3  
1               No explanation found                                           NaN  
2               No explanation found                                           NaN  
7               No explanation found                                           NaN  
12              No explanation found                                           NaN

### Summary of results

In [147]:
conj_averages = spotify_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = spotify_comparison_df.loc[:, 'disjunctive', :].mean()
spotify_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
spotify_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [148]:
spotify_comparison_df

Average Coverage  \
Test                     Explanation Type                      
Full dataset             conjunctive      0.8833333333333333   
                         disjunctive      0.9711111111111111   
Numeric only             conjunctive      0.9424999999999999   
                         disjunctive      0.9744444444444444   
Categorical only         conjunctive                     1.0   
                         disjunctive                     1.0   
Single attribute groupby conjunctive                    0.82   
                         disjunctive                    0.91   
3 clusters               conjunctive      0.9960000000000001   
                         disjunctive      0.9960000000000001   
15 clusters              conjunctive      0.9182456140350876   
                         disjunctive      0.9800000000000001   
Average                  conjunctive      0.9266798245614035   
                         disjunctive       0.971925925925926   

                                          Average Separation Error  \
Test                     Explanation Type                            
Full dataset             conjunctive                         0.035   
                         disjunctive          0.049999999999999996   
Numeric only             conjunctive                         0.045   
                         disjunctive           0.05888888888888889   
Categorical only         conjunctive                          0.07   
                         disjunctive                          0.07   
Single attribute groupby conjunctive            0.2833333333333334   
                         disjunctive            0.2933333333333334   
3 clusters               conjunctive                         0.016   
                         disjunctive                         0.016   
15 clusters              conjunctive           0.14578947368421047   
                         disjunctive           0.21599999999999997   
Average                  conjunctive           0.09918713450292398   
                         disjunctive           0.11737037037037039   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Full dataset             conjunctive                         50.0   
                         disjunctive                         50.0   
Numeric only             conjunctive                         50.0   
                         disjunctive                         50.0   
Categorical only         conjunctive                         50.0   
                         disjunctive                         50.0   
Single attribute groupby conjunctive                         50.0   
                         disjunctive                         50.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                         10.0   
                         disjunctive                         40.0   
Average                  conjunctive                         35.0   
                         disjunctive                         40.0   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Full dataset             conjunctive              1.3333333333333333   
                         disjunctive              1.5555555555555556   
Numeric only             conjunctive                             1.0   
                         disjunctive              1.5555555555555556   
Categorical only         conjunctive                             1.0   
                         disjunctive                             1.0   
Single attribute groupby conjunctive              1.3333333333333333   
                         disjunctive              1.3333333333333333   
3 clusters               conjunctive                             1.0   
 

## House Prices dataset

In [7]:
houses = pd.read_csv(r"..\Examples\Datasets\houses.csv")
houses = houses[['MSSubClass', 'LotArea', 'Street', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle',
'OverallQual', 'OverallCond', 'YearBuilt', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'ExterQual', 'Foundation', 'BsmtQual',
'CentralAir', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'PoolArea', 'YrSold', 'SalePrice']]

In [8]:
houses_comparison_df = pd.DataFrame(columns=columns, index=index)

### Full dataset test

In [151]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels='HouseStyle', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Full dataset', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 5.878854990005493


Coverage  \
Group / Cluster Explanation                                                                  
2Story          600 <= 2ndFlrSF <= 2065                                               0.91   
                55.0 <= MSSubClass <= 72.5                                             0.8   
                545.5 <= 2ndFlrSF <= 2065                                             0.97   
                200.0 <= 2ndFlrSF <= 2065                                              1.0   
                545.5 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass <= 190                 0.96   
                200.0 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass <= 190                 0.99   
                671.0 <= 2ndFlrSF <= 2065 AND 60 <= MSSubClass <= 190                 0.85   
                671.0 <= 2ndFlrSF <= 2065                                             0.85   
                545.5 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClass <= 190               0.96   
                671.0 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClass <= 190               0.85   
                200.0 <= 2ndFlrSF <= 2065 AND 55.0 <= MSSubClass <= 190               0.99   
1Story          2ndFlrSF == 0 AND 20 <= MSSubClass <= 30                              0.83   
                0 <= 2ndFlrSF <= 545.5 AND 20 <= MSSubClass <= 30                     0.83   
                MSSubClass == 20                                                      0.74   
                0 <= 2ndFlrSF <= 728 AND 20 <= MSSubClass <= 42.5                     0.83   
                0 <= 2ndFlrSF <= 55.0 AND 20 <= MSSubClass <= 30                      0.83   
                20 <= MSSubClass <= 42.5                                              0.83   
                2ndFlrSF == 0 AND 20 <= MSSubClass <= 42.5                            0.83   
                0 <= 2ndFlrSF <= 545.5 AND 20 <= MSSubClass <= 42.5                   0.83   
                0 <= 2ndFlrSF <= 55.0 AND 20 <= MSSubClass <= 42.5                    0.83   
                2ndFlrSF == 0                                                          1.0   
                0 <= 2ndFlrSF <= 600 AND 20 <= MSSubClass <= 30                       0.83   
                0.0 <= 2ndFlrSF <= 413.0 AND 20 <= MSSubClass <= 30                   0.83   
                0.0 <= 2ndFlrSF <= 516.25 AND 20 <= MSSubClass <= 30                  0.83   
                0 <= 2ndFlrSF <= 600 AND 20 <= MSSubClass <= 42.5                     0.83   
                0.0 <= 2ndFlrSF <= 413.0 AND 20 <= MSSubClass <= 42.5                 0.83   
                0.0 <= 2ndFlrSF <= 344.1666666666667 AND 20 <= MSSubClass <= 30       0.83   
                0.0 <= 2ndFlrSF <= 516.25 AND 20 <= MSSubClass <= 42.5                0.83   
                20 <= MSSubClass <= 25.0                                              0.74   
                0 <= 2ndFlrSF <= 456 AND 20 <= MSSubClass <= 30                       0.83   
                0.0 <= 2ndFlrSF <= 688.3333333333334 AND 20 <= MSSubClass <= 30       0.83   
                2ndFlrSF == 0                                                          1.0   
                0.0 <= 2ndFlrSF <= 344.1666666666667 AND 20 <= MSSubClass <= 42.5     0.83   
                2ndFlrSF == 0 AND 766.0 <= 1stFlrSF <= 4692                           0.97   
                0 <= 2ndFlrSF <= 456 AND 20 <= MSSubClass <= 42.5                     0.83   
                0 <= 2ndFlrSF <= 55.0 AND 766.0 <= 1stFlrSF <= 4692                   0.97   
                0.0 <= 2ndFlrSF <= 688.3333333333334 AND 20 <= MSSubClass <= 42.5     0.83   
                0 <= 2ndFlrSF <= 728 AND 20 <= MSSubClass <= 30                       0.83   
                20 <= MSSubClass <= 30                                                0.83   
                0 <= 2ndFlrSF <= 55.0                                                  1.0   
                MSSubClass == 20                                                      0.74   
1.5Fin          MSSubClass == 50        

In [152]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels='HouseStyle', explanation_form='disj', bin_numeric=True)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Full dataset', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 3.243934392929077


Coverage  \
Group / Cluster Explanation                                                        
2Story          55.0 <= MSSubClass <= 72.5                                   0.8   
                600 <= 2ndFlrSF <= 2065                                     0.91   
                671.0 <= 2ndFlrSF <= 2065                                   0.85   
                545.5 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass <= 72.5     0.98   
                200.0 <= 2ndFlrSF <= 2065                                    1.0   
                55.0 <= MSSubClass <= 72.5 OR 671.0 <= 2ndFlrSF <= 2065     0.91   
                600 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass <= 72.5       0.95   
                545.5 <= 2ndFlrSF <= 2065                                   0.97   
1Story          20 <= MSSubClass <= 30                                      0.83   
                0 <= 2ndFlrSF <= 0                                           1.0   
                20 <= MSSubClass <= 20                                      0.74   
                20 <= MSSubClass <= 42.5                                    0.83   
                20 <= MSSubClass <= 25.0                                    0.74   
                0 <= 2ndFlrSF <= 55.0                                        1.0   
1.5Fin          42.5 <= MSSubClass <= 55.0                                  0.92   
1.5Unf          42.5 <= MSSubClass <= 47.5                                  0.86   
2.5Fin          No explanation found                                         NaN   
2.5Unf          No explanation found                                         NaN   
SFoyer          No explanation found                                         NaN   
SLvl            No explanation found                                         NaN   

                                                                        Separation Error  
Group / Cluster Explanation                                                               
2Story          55.0 <= MSSubClass <= 72.5                                          0.01  
                600 <= 2ndFlrSF <= 2065                                             0.17  
                671.0 <= 2ndFlrSF <= 2065                                           0.14  
                545.5 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass <= 72.5              0.2  
                200.0 <= 2ndFlrSF <= 2065                                           0.29  
                55.0 <= MSSubClass <= 72.5 OR 671.0 <= 2ndFlrSF <= 2065             0.13  
                600 <= 2ndFlrSF <= 2065 OR 55.0 <= MSSubClass <= 72.5               0.17  
                545.5 <= 2ndFlrSF <= 2065                                            0.2  
1Story          20 <= MSSubClass <= 30                                              0.01  
                0 <= 2ndFlrSF <= 0                                                  0.13  
                20 <= MSSubClass <= 20                                               0.0  
                20 <= MSSubClass <= 42.5                                            0.01  
                20 <= MSSubClass <= 25.0                                             0.0  
                0 <= 2ndFlrSF <= 55.0                                               0.13  
1.5Fin          42.5 <= MSSubClass <= 55.0                                           0.1  
1.5Unf          42.5 <= MSSubClass <= 47.5                                           0.0  
2.5Fin          No explanation found                                                 NaN  
2.5Unf          No explanation found                                                 NaN  
SFoyer          No explanation found                                                 NaN  
SLvl            No explanation found                                                 NaN

### Numeric only and categorical only tests

In [7]:
numeric_only = houses[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'PoolArea', 'SalePrice']]
categorical_only = houses[['MSSubClass', 'Street', 'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'ExterQual', 'Foundation', 'BsmtQual', 'CentralAir', 'YrSold']]
labels = houses['HouseStyle']

NameError: name 'houses' is not defined

In [154]:
start_time = time.time()
conj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Numeric only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 5.5691399574279785


Coverage  \
Group / Cluster Explanation                                                                                
2Story          545.5 <= 2ndFlrSF <= 2065 AND 1941.0 <= YearBuilt <= 2010.0                         0.82   
                545.5 <= 2ndFlrSF <= 2065 AND 1968.5 <= YearBuilt <= 2010                           0.75   
                600 <= 2ndFlrSF <= 2065                                                             0.91   
                671.0 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivArea <= 2988.0                          0.83   
                671.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBuilt <= 2010.0                          0.7   
                545.5 <= 2ndFlrSF <= 2065                                                           0.97   
                200.0 <= 2ndFlrSF <= 2065                                                            1.0   
                200.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBuilt <= 2010.0                         0.81   
                545.5 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivArea <= 2988.0                          0.95   
                671.0 <= 2ndFlrSF <= 2065                                                           0.85   
                688.3333333333334 <= 2ndFlrSF <= 1376.6666666666667                                 0.78   
                545.5 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBuilt <= 2010.0                         0.79   
1Story          0 <= 2ndFlrSF <= 55.0 AND 2776.0 <= LotArea <= 215245                                1.0   
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245                                        1.0   
                2ndFlrSF == 0                                                                        1.0   
                2ndFlrSF == 0                                                                        1.0   
                0 <= 2ndFlrSF <= 55.0                                                                1.0   
                2ndFlrSF == 0 AND 2776.0 <= LotArea <= 215245 AND 1941.0 <= YearBuilt <= 2010.0     0.91   
1.5Fin          No explanation found                                                                 NaN   
1.5Unf          No explanation found                                                                 NaN   
2.5Fin          No explanation found                                                                 NaN   
2.5Unf          No explanation found                                                                 NaN   
SFoyer          No explanation found                                                                 NaN   
SLvl            No explanation found                                                                 NaN   

                                                                                                Separation Error  
Group / Cluster Explanation                                                                                       
2Story          545.5 <= 2ndFlrSF <= 2065 AND 1941.0 <= YearBuilt <= 2010.0                                 0.12  
                545.5 <= 2ndFlrSF <= 2065 AND 1968.5 <= YearBuilt <= 2010                                   0.06  
                600 <= 2ndFlrSF <= 2065                                                                     0.17  
                671.0 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivArea <= 2988.0                                  0.13  
                671.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBuilt <= 2010.0                                 0.04  
                545.5 <= 2ndFlrSF <= 2065                                                                    0.2  
                200.0 <= 2ndFlrSF <= 2065                                                                   0.29  
                200.0 <= 2ndFlrSF <= 2065 AND 1964.0 <= YearBuilt <= 2010.0                                 0.08  
                545.5 <= 2ndFlrSF <= 2065 AND 334.0 <= GrLivArea <= 2988.0                                  0.19  
                671.0 

In [8]:
start_time = time.time()
disj_res = numeric_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Numeric only', 'disjunctive', disj_res, disj_time)
disj_res

NameError: name 'numeric_only' is not defined

In [156]:
start_time = time.time()
conj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Categorical only', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.4885356426239014


Coverage Separation Error
Group / Cluster Explanation                                         
2Story          50 <= MSSubClass <= 70         0.81             0.28
                55.0 <= MSSubClass <= 72.5      0.8             0.01
1Story          MSSubClass == 20               0.74              0.0
                20 <= MSSubClass <= 42.5       0.83             0.01
                20 <= MSSubClass <= 30         0.83             0.01
                20 <= MSSubClass <= 25.0       0.74              0.0
                MSSubClass == 20               0.74              0.0
1.5Fin          MSSubClass == 50               0.92             0.02
1.5Unf          MSSubClass == 45               0.86              0.0
                42.5 <= MSSubClass <= 47.5     0.86              0.0
SLvl            MSSubClass == 80               0.89              0.0
2.5Fin          No explanation found            NaN              NaN
2.5Unf          No explanation found            NaN              NaN
SFoyer          No explanation found            NaN              NaN

In [157]:
start_time = time.time()
disj_res = categorical_only.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Categorical only', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.4473304748535156


Coverage Separation Error
Group / Cluster Explanation                                         
2Story          55.0 <= MSSubClass <= 72.5      0.8             0.01
                50 <= MSSubClass <= 70         0.81             0.28
1Story          20 <= MSSubClass <= 20         0.74              0.0
                20 <= MSSubClass <= 25.0       0.74              0.0
                20 <= MSSubClass <= 42.5       0.83             0.01
                20 <= MSSubClass <= 30         0.83             0.01
1.5Fin          42.5 <= MSSubClass <= 55.0     0.92              0.1
1.5Unf          42.5 <= MSSubClass <= 47.5     0.86              0.0
2.5Fin          No explanation found            NaN              NaN
2.5Unf          No explanation found            NaN              NaN
SFoyer          No explanation found            NaN              NaN
SLvl            No explanation found            NaN              NaN

### Groupby tests

In [14]:
houses

,MSSubClass,LotArea,Street,LotConfig,LandSlope,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,...,CentralAir,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,PoolArea,YrSold,SalePrice
0,60,8450,Pave,Inside,Gtl,CollgCr,1Fam,2Story,7,5,...,Y,856,854,1710,2,8,2,0,2008,208500
1,20,9600,Pave,FR2,Gtl,Veenker,1Fam,1Story,6,8,...,Y,1262,0,1262,2,6,2,0,2007,181500
2,60,11250,Pave,Inside,Gtl,CollgCr,1Fam,2Story,7,5,...,Y,920,866,1786,2,6,2,0,2008,223500
3,70,9550,Pave,Corner,Gtl,Crawfor,1Fam,2Story,7,5,...,Y,961,756,1717,1,7,3,0,2006,140000
4,60,14260,Pave,FR2,Gtl,NoRidge,1Fam,2Story,8,5,...,Y,1145,1053,2198,2,9,3,0,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,Pave,Inside,Gtl,Gilbert,1Fam,2Story,6,5,...,Y,953,694,1647,2,7,2,0,2007,175000
1456,20,13175,Pave,Inside,Gtl,NWAmes,1Fam,1Story,6,6,...,Y,2073,0,2073,2,7,2,0,2010,210000
1457,70,9042,Pave,Inside,Gtl,Crawfor,1Fam,2Story,7,9,...,Y,1188,1152,2340,2,9,1,0,2010,266500
1458,20,9717,Pave,Inside,Gtl,NAmes,1Fam,1Story,5,6,...,Y,1078,0,1078,1,5,1,0,2010,142125


In [34]:
single_attribute_groupby = houses.groupby('ExterQual').mean()
multi_attribute_groupby = houses.groupby(['OverallQual', 'ExterQual']).mean()

In [10]:
start_time = time.time()
conj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Single attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.1962735652923584


Coverage  \
Group / Cluster Explanation                                                   
Gd              1980.5 <= YearBuilt <= 2010 AND 6.4 <= OverallQ...     0.72   
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...     0.72   
                1982.4 <= YearBuilt <= 2010.0                          0.87   
                1964.0 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.74   
                7.0 <= OverallQual <= 8.5                              0.76   
                1987.0 <= YearBuilt <= 2010.0 AND 5.5 <= Overal...     0.85   
                1982.4 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.72   
                1980.5 <= YearBuilt <= 2010 AND 5.5 <= OverallQ...     0.85   
                1980.5 <= YearBuilt <= 2010 AND 5.5 <= OverallQ...     0.88   
                1982.4 <= YearBuilt <= 2010.0 AND 6.5 <= Overal...     0.76   
                6.4 <= OverallQual <= 8.2                              0.76   
                1984.5 <= YearBuilt <= 2010                            0.86   
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...     0.76   
                1975.5 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.73   
                1975.5 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.77   
                1987.0 <= YearBuilt <= 2010.0 AND 5.5 <= Overal...     0.82   
                1980.5 <= YearBuilt <= 2010                            0.88   
                1980.5 <= YearBuilt <= 2010 AND 6 <= OverallQua...     0.85   
                1980.5 <= YearBuilt <= 2010 AND 171625.0 <= Sal...     0.79   
                1975.5 <= YearBuilt <= 2010.0 AND 6.5 <= Overal...     0.77   
                1964.0 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.74   
                1987.0 <= YearBuilt <= 2010.0 AND 6 <= OverallQ...     0.82   
                1987.0 <= YearBuilt <= 2010.0 AND 6 <= OverallQ...     0.85   
                1982.4 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.72   
                1975.5 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...     0.73   
                1982.4 <= YearBuilt <= 2010.0 AND 7.0 <= Overal...     0.76   
                1980.5 <= YearBuilt <= 2010 AND 6.5 <= OverallQ...     0.74   
                1980.5 <= YearBuilt <= 2010 AND 6.5 <= OverallQ...     0.76   
                1980.5 <= YearBuilt <= 2010 AND 6 <= OverallQua...     0.88   
TA              4.0 <= OverallQual <= 7.0                              0.97   
                1 <= OverallQual <= 6                                  0.89   
                1.5 <= OverallQual <= 6.5                              0.89   
                1872 <= YearBuilt <= 1980.5                            0.86   
                4.0 <= OverallQual <= 7.0 AND 1872 <= YearBuilt...     0.84   
                1 <= OverallQual <= 6.5                                0.89   
                1.5 <= OverallQual <= 6.5 AND 1872 <= YearBuilt...     0.79   
Ex              No explanation found                                    NaN   
Fa              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Gd              1980.5 <= YearBuilt <= 2010 AND 6.4 <= OverallQ...             0.12  
                1980.5 <= YearBuilt <= 2010 AND 7.0 <= OverallQ...             0.12  
                1982.4 <= YearBuilt <= 2010.0                                  0.28  
                1964.0 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...             0.18  
                7.0 <= OverallQual <= 8.5                                      0.24  
                1987.0 <= YearBuilt <= 2010.0 AND 5.5 <= Overal...             0.24  
                1982.4 <= YearBuilt <= 2010.0 AND 6.4 <= Overal...             0.12  
                1980.5 <= YearBuilt <= 2010 AND 5.5 <= OverallQ...             0.23  
                1980.5 <= YearBuilt <= 2010 AN

In [18]:
start_time = time.time()
disj_res = single_attribute_groupby.explain(explainer='many to one', explanation_form='disj', max_labels=50)
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Single attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 0.9243829250335693


Coverage  \
Group / Cluster Explanation                                                   
Gd              1982.4 <= YearBuilt <= 2010.0                          0.87   
                6.4 <= OverallQual <= 8.2                              0.76   
                1980.5 <= YearBuilt <= 2010                            0.88   
                1984.5 <= YearBuilt <= 2010                            0.86   
                7.0 <= OverallQual <= 8.5                              0.76   
TA              1872 <= YearBuilt <= 1980.5 OR 1 <= OverallQual...     0.96   
                1872 <= YearBuilt <= 1980.5 OR 1 <= OverallQual...     0.96   
                1 <= OverallQual <= 6                                  0.89   
                1 <= OverallQual <= 6.5                                0.89   
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...      1.0   
                1872 <= YearBuilt <= 1980.5 OR 1.5 <= OverallQu...     0.96   
                1.5 <= OverallQual <= 6.5                              0.89   
                4.0 <= OverallQual <= 7.0                              0.97   
                1872 <= YearBuilt <= 1980.5                            0.86   
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...      1.0   
Ex              No explanation found                                    NaN   
Fa              No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Gd              1982.4 <= YearBuilt <= 2010.0                                  0.28  
                6.4 <= OverallQual <= 8.2                                      0.24  
                1980.5 <= YearBuilt <= 2010                                    0.29  
                1984.5 <= YearBuilt <= 2010                                    0.27  
                7.0 <= OverallQual <= 8.5                                      0.24  
TA              1872 <= YearBuilt <= 1980.5 OR 1 <= OverallQual...             0.14  
                1872 <= YearBuilt <= 1980.5 OR 1 <= OverallQual...             0.14  
                1 <= OverallQual <= 6                                          0.12  
                1 <= OverallQual <= 6.5                                        0.12  
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...             0.27  
                1872 <= YearBuilt <= 1980.5 OR 1.5 <= OverallQu...             0.14  
                1.5 <= OverallQual <= 6.5                                      0.12  
                4.0 <= OverallQual <= 7.0                                      0.27  
                1872 <= YearBuilt <= 1980.5                                    0.09  
                4.0 <= OverallQual <= 7.0 OR 1872 <= YearBuilt ...             0.27  
Ex              No explanation found                                            NaN  
Fa              No explanation found                                            NaN

In [35]:
start_time = time.time()
conj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, 'Multi attribute groupby', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 3.0623204708099365


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"(4, 'TA')",No explanation found,NaN,NaN
"(5, 'Gd')",No explanation found,NaN,NaN
"(5, 'TA')",No explanation found,NaN,NaN
"(6, 'Gd')",No explanation found,NaN,NaN
"(6, 'TA')",No explanation found,NaN,NaN
"(7, 'Gd')",No explanation found,NaN,NaN
"(7, 'TA')",No explanation found,NaN,NaN
"(8, 'Gd')",No explanation found,NaN,NaN
"(9, 'Ex')",No explanation found,NaN,NaN


In [36]:
start_time = time.time()
disj_res = multi_attribute_groupby.explain(explainer='many to one', explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, 'Multi attribute groupby', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 2.44771671295166


,,Coverage,Separation Error
Group / Cluster,Explanation,,
"(4, 'TA')",No explanation found,NaN,NaN
"(5, 'Gd')",No explanation found,NaN,NaN
"(5, 'TA')",No explanation found,NaN,NaN
"(6, 'Gd')",No explanation found,NaN,NaN
"(6, 'TA')",No explanation found,NaN,NaN
"(7, 'Gd')",No explanation found,NaN,NaN
"(7, 'TA')",No explanation found,NaN,NaN
"(8, 'Gd')",No explanation found,NaN,NaN
"(9, 'Ex')",No explanation found,NaN,NaN


### Clustering test

#### 3 clusters

In [37]:
labels = KMeans(n_clusters=3).fit_predict(pd.get_dummies(houses))

C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  Fi

In [38]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, '3 clusters', 'conjunctive', conj_res, conj_time)
conj_res

Conjunctive explanation runtime: 1.9508733749389648


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,293538.5 <= SalePrice <= 755000,1.0,0.0
1,34900 <= SalePrice <= 172200.0,1.0,0.0
2,172200.0 <= SalePrice <= 293538.5,1.0,0.0


In [39]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, '3 clusters', 'disjunctive', disj_res, disj_time)
disj_res

Disjunctive explanation runtime: 1.61720871925354


,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,293538.5 <= SalePrice <= 755000,1.0,0.0
1,34900 <= SalePrice <= 172200.0,1.0,0.0
2,172200.0 <= SalePrice <= 293538.5,1.0,0.0


#### 15 clusters

In [40]:
labels = KMeans(n_clusters=15).fit_predict(pd.get_dummies(houses))

In [41]:
start_time = time.time()
conj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='conj')
conj_time = time.time() - start_time
print(f"Conjunctive explanation runtime: {conj_time}")
build_comparison_df(houses_comparison_df, '15 clusters', 'conjunctive', conj_res, conj_time)
conj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Conjunctive explanation runtime: 5.602252721786499


Coverage  \
Group / Cluster Explanation                                                   
0               301865.3333333334 <= SalePrice <= 358581.666666...     0.95   
1               148650.0 <= SalePrice <= 170500.0                      0.99   
3               75000 <= SalePrice <= 102388.0                          1.0   
6               102388.0 <= SalePrice <= 126337.5                       1.0   
7               245149.0 <= SalePrice <= 301865.3333333334 AND ...      0.9   
                245149.0 <= SalePrice <= 301865.3333333334              1.0   
11              358581.6666666667 <= SalePrice <= 415298.0              1.0   
                356550.0 <= SalePrice <= 415298                         1.0   
13              170500.0 <= SalePrice <= 195200.0                       1.0   
14              126337.5 <= SalePrice <= 148650.0                       1.0   
2               No explanation found                                    NaN   
5               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               301865.3333333334 <= SalePrice <= 358581.666666...              0.0  
1               148650.0 <= SalePrice <= 170500.0                               0.0  
3               75000 <= SalePrice <= 102388.0                                  0.0  
6               102388.0 <= SalePrice <= 126337.5                               0.0  
7               245149.0 <= SalePrice <= 301865.3333333334 AND ...             0.22  
                245149.0 <= SalePrice <= 301865.3333333334                     0.24  
11              358581.6666666667 <= SalePrice <= 415298.0                      0.0  
                356550.0 <= SalePrice <= 415298                                 0.0  
13              170500.0 <= SalePrice <= 195200.0                               0.0  
14              126337.5 <= SalePrice <= 148650.0                               0.0  
2               No explanation found                                            NaN  
5               No explanation found                                            NaN

In [42]:
start_time = time.time()
disj_res = houses.explain(explainer='many to one', labels=labels, explanation_form='disj')
disj_time = time.time() - start_time
print(f"Disjunctive explanation runtime: {disj_time}")
build_comparison_df(houses_comparison_df, '15 clusters', 'disjunctive', disj_res, disj_time)
disj_res

There are more than the specified max number of 10 unique labels, and the option `prune_if_too_many_labels` is set to True. Pruning the labels to the top 10 most common labels.

Disjunctive explanation runtime: 4.3408918380737305


Coverage  \
Group / Cluster Explanation                                                   
0               301865.3333333334 <= SalePrice <= 358581.666666...     0.95   
1               148650.0 <= SalePrice <= 170500.0                      0.99   
3               75000 <= SalePrice <= 102388.0                          1.0   
6               102388.0 <= SalePrice <= 126337.5                       1.0   
7               245149.0 <= SalePrice <= 301865.3333333334              1.0   
11              356550.0 <= SalePrice <= 415298                         1.0   
                358581.6666666667 <= SalePrice <= 415298.0              1.0   
13              170500.0 <= SalePrice <= 195200.0                       1.0   
14              126337.5 <= SalePrice <= 148650.0                       1.0   
2               No explanation found                                    NaN   
5               No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               301865.3333333334 <= SalePrice <= 358581.666666...              0.0  
1               148650.0 <= SalePrice <= 170500.0                               0.0  
3               75000 <= SalePrice <= 102388.0                                  0.0  
6               102388.0 <= SalePrice <= 126337.5                               0.0  
7               245149.0 <= SalePrice <= 301865.3333333334                     0.24  
11              356550.0 <= SalePrice <= 415298                                 0.0  
                358581.6666666667 <= SalePrice <= 415298.0                      0.0  
13              170500.0 <= SalePrice <= 195200.0                               0.0  
14              126337.5 <= SalePrice <= 148650.0                               0.0  
2               No explanation found                                            NaN  
5               No explanation found                                            NaN

### Summary of results

In [43]:
conj_averages = houses_comparison_df.loc[:, 'conjunctive', :].mean()
disj_averages = houses_comparison_df.loc[:, 'disjunctive', :].mean()
houses_comparison_df.loc[('Average', 'conjunctive'), :] = conj_averages
houses_comparison_df.loc[('Average', 'disjunctive'), :] = disj_averages

In [44]:
houses_comparison_df

Average Coverage  \
Test                     Explanation Type                    
Single attribute groupby conjunctive              0.805278   
                         disjunctive              0.900667   
Multi attribute groupby  conjunctive                   NaN   
                         disjunctive                   NaN   
3 clusters               conjunctive                   1.0   
                         disjunctive                   1.0   
15 clusters              conjunctive                 0.984   
                         disjunctive              0.993333   
Average                  conjunctive              0.929759   
                         disjunctive              0.964667   

                                          Average Separation Error  \
Test                     Explanation Type                            
Single attribute groupby conjunctive                      0.187222   
                         disjunctive                           0.2   
Multi attribute groupby  conjunctive                           NaN   
                         disjunctive                           NaN   
3 clusters               conjunctive                           0.0   
                         disjunctive                           0.0   
15 clusters              conjunctive                         0.046   
                         disjunctive                      0.026667   
Average                  conjunctive                      0.077741   
                         disjunctive                      0.075556   

                                          % Explanation Not Found  \
Test                     Explanation Type                           
Single attribute groupby conjunctive                         50.0   
                         disjunctive                         50.0   
Multi attribute groupby  conjunctive                        100.0   
                         disjunctive                        100.0   
3 clusters               conjunctive                            0   
                         disjunctive                            0   
15 clusters              conjunctive                         20.0   
                         disjunctive                         20.0   
Average                  conjunctive                         42.5   
                         disjunctive                         42.5   

                                          Average Explanation Length  \
Test                     Explanation Type                              
Single attribute groupby conjunctive                        1.861111   
                         disjunctive                        1.333333   
Multi attribute groupby  conjunctive                             NaN   
                         disjunctive                             NaN   
3 clusters               conjunctive                             1.0   
                         disjunctive                             1.0   
15 clusters              conjunctive                             1.1   
                         disjunctive                             1.0   
Average                  conjunctive                         1.32037   
                         disjunctive                        1.111111   

                                          Number of Explanations Found  \
Test                     Explanation Type                                
Single attribute groupby conjunctive                                36   
                         disjunctive                                15   
Multi attribute groupby  conjunctive                                 0   
                         disjunctive                                 0   
3 clusters               conjunctive                                 3   
                         disjunctive                                 3   
15 clusters              conjunctive                                10   
                         disjunctive                                 9   
Average               